In [1]:
import pandas as pd
import numpy as np
import random
import json
from collections import defaultdict
from sklearn import datasets
from multiprocessing.dummy import Pool
from datetime import datetime

# Pre-processing

In [ ]:
covid = pd.read_csv('Dataset_Covid_CE.csv')

In [ ]:
covid = covid[['estadoCaso', 'codigoMunicipioCaso', 'municipioCaso', 'bairroCaso', 'sexoCaso', 'dataNascimento', 
               'idadeCaso','racaCor', 'resultadoFinalExame']]
global df
df = covid
df.dropna(inplace=True)

In [ ]:
df.head()

### Pegando as mesoregiões para generalizar a localidade em mais um nível

In [ ]:
regioes = pd.read_csv('CE_region.csv') # database from IpeaGEO
regioes = regioes[['Código IBGE do Município', 'Nome da Mesoregião']]
regioes['Nome da Mesoregião'].replace('Sert§es Cearenses', 'Sertões Cearenses', inplace=True)

In [ ]:
regioes.head(5)

In [ ]:
dict_meso = defaultdict(list)
for meso in regioes['Nome da Macroregião'].unique():
    dict_meso[meso] = list(regioes['Código IBGE do Município'][regioes['Nome da Mesoregião'] == meso])

In [ ]:
mesoregioes = []
for i in range(len(df)):
    for key in dict_meso.keys():
        if(df['codigoMunicipioCaso'].iloc[i] in dict_meso[key]):
            mesoregioes.append(key)
df['mesoregiaoCaso'] = mesoregioes
df = df[['estadoCaso', 'codigoMunicipioCaso', 'municipioCaso', 'mesoregiaoCaso', 'bairroCaso', 'sexoCaso', 'dataNascimento', 
         'idadeCaso','racaCor', 'resultadoFinalExame']]

In [ ]:
df

In [ ]:
df.head()

### Joining the three locality attributes to construct a generalization tree

In [ ]:
local = []
for i in range(len(df)):
    local.append('{}; {}; {}; {}'.format(df['bairroCaso'].iloc[i], df['municipioCaso'].iloc[i], df['mesoregiaoCaso'].iloc[i], df['estadoCaso'].iloc[i]))
    # changing date format
    df['dataNascimento'].iloc[i] = '{}/{}/{}'.format(df['dataNascimento'].iloc[i][-2:], 
                                                     df['dataNascimento'].iloc[i][5:7],
                                                     df['dataNascimento'].iloc[i][:4])
    
df = df.drop(['estadoCaso', 'mesoregiaoCaso', 'municipioCaso', 'bairroCaso'], axis=1)
df['localCaso'] = local
df = df[['localCaso', 'sexoCaso', 'dataNascimento', 'idadeCaso', 'racaCor', 'resultadoFinalExame']]

df.head()

In [ ]:
df.to_csv('covid_com_meso.csv')

# Just call the already pre-processed dataset

In [2]:
df = pd.read_csv('covid_com_meso.csv')
df.drop(columns='Unnamed: 0', axis=1, inplace=True)
global df

# Suppression

In [3]:
def suppression(features):
    for feature in features:
        dataset[feature].replace(dataset[feature].unique(), '*', inplace=True)
    return dataset

# Generalization

In [4]:
def generalization(level_local=0, level_bday=0):
    dataset = generalizing_local(level_local)
    dataset = generalizing_bday(dataset, level_bday)
    
    return dataset

### Generalizing County

In [5]:
def generalizing_local(level=0):
    if(level == 1):
        for i in range(len(dataset)):
            neigh, county, m_region, state = dataset['localCaso'].iloc[i].split('; ')
            dataset['localCaso'].iloc[i] = '*; {}; {}; {}'.format(county, m_region, state)
    elif(level == 2):
        for i in range(len(dataset)):
            neigh, county, m_region, state = dataset['localCaso'].iloc[i].split('; ')
            dataset['localCaso'].iloc[i] = '*; *; {}; {}'.format(m_region, state)
    elif(level == 3):
        for i in range(len(dataset)):
            neigh, county, m_region, state = dataset['localCaso'].iloc[i].split('; ')
            dataset['localCaso'].iloc[i] = '*; *; *; {}'.format(state)
    
    return dataset

### Generalizing birthday

In [6]:
def generalizing_bday(dataset, level=0):
    dataset = dataset.copy()
    dataset = dataset
    if(level == 1):
        for i in range(len(dataset)):
            day, month, year = dataset['dataNascimento'].iloc[i].split('/')
            dataset['dataNascimento'].iloc[i] = '*/{}/{}'.format(month, year)
    elif(level == 2):
        for i in range(len(dataset)):
            day, month, year = dataset['dataNascimento'].iloc[i].split('/')
            dataset['dataNascimento'].iloc[i] = '*/*/{}'.format(year)
    
    return dataset

# Perturbation

In [7]:
from scipy.stats import laplace

In [8]:
def adding_laplace_noise(index, value):
    eps_n = eps/len(dataset)
    noise = laplace.rvs(loc=0, scale = sensitivity/eps_n)
    value = np.ceil(round(value + noise, 1)) # remove ceil(*) for float values\n",
   
    return index, value

In [9]:
def perturbation(feature):
    global sensitivity
    global eps
    
    if(isinstance(dataset[feature].iloc[0], str)):
        values, counts = dataset[feature].value_counts().index, dataset[feature].value_counts().values
        dataset[feature] = random.choices(values, weights=counts, k=len(dataset[feature])) # following the real frequencies distribution
    else:
        index_feature_2D = np.c_[np.array(dataset.index), np.array(dataset[feature])]
        eps = 75*len(dataset)
        sensitivity = max(dataset[feature]) - min(dataset[feature])
        with Pool(15) as pool:
            threads = pool.starmap(adding_laplace_noise, index_feature_2D)
            pool.terminate()
            pool.close()
        pool.join()
        
        dataset[feature] = [value for value in list(dict(threads).values())]
    
    return dataset

### Testing Perturbation on COVID data

Tamanho do dataset:  825977
Para quantas amostras deseja executar as operações?
10000


C:\Users\malu.maia\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
def menu():
    u = '#'*15
    print('\n\n{} Menu {}\n\n1 - Carregar dados pré-processados\n2 - Supressão\n3 - Generalização\n4 - Perturbação\n5 - Visualizar dados anonimizados\n6 - Calcular estatísticas pós-perturbação\n7 - Sair\n\n'.format(u, u))
    op = input('\nDigite a operação que deseja realizar:\n')
    return op

def op_1():
    print(pre_df)

def op_2():
    def menu_features():
        print('Selecione os atributos que deseja suprimir (separe os atributos com vírgula+espaço: ", "):\
               \n1 - Local\n2 - Gênero\n3 - Data do nascimento\n4 - Idade\n5 - Raça/Cor\n6 - Resultado do exame\n\n')
        features = input()
        return features
    
    global dataset
    features = menu_features()
    features = features.split(', ')
    attr = {'1': 'localCaso',
            '2': 'sexoCaso',
            '3': 'dataNascimento',
            '4': 'idadeCaso',
            '5': 'racaCor',
            '6': 'resultadoFinalExame'
           }
    
    flag = True
    for feat in features:
        if(feat not in attr.keys()):
            flag = False
    if(flag == True):
        dataset = suppression([attr[i] for i in features])
        print(dataset)
    else:
        print("\nValores inválidos.\n")
        op_2()
        
def op_3():
    global dataset
    print('\nSelecione o nível de generalização desejado para local, com 0 sendo o menor nivel e 3 o maior:\n')
    level_local = int(input())
    print('\nAgora selecione o nível de generalização desejado para nascimento, com 0 sendo o menor nível e 2 o maior:\n')
    level_bday = int(input())
    dataset = generalization(level_local, level_bday)
    print(dataset)
    
def op_4():
    def menu_feature():
        print('Selecione o atributo que deseja aplicar a perturbação:\
               \n1 - Local\n2 - Gênero\n3 - Data do nascimento\n4 - Idade\n5 - Raça/Cor\n6 - Resultado do exame\n\n')
        feature = input()
        return feature
    
    global dataset
    feature = menu_feature()
    attr = {'1': 'localCaso',
            '2': 'sexoCaso',
            '3': 'dataNascimento',
            '4': 'idadeCaso',
            '5': 'racaCor',
            '6': 'resultadoFinalExame'
           }
    
    dataset = perturbation(attr[feature])
    print(dataset)
    

def options():
    op = menu()

    if(op == '1'):
        op_1()
        options()

    elif(op == '2'):
        op_2()
        options()

    elif(op == '3'):
        op_3()
        options()
        
    elif(op == '4'):
        op_4()
        options()
    
    elif(op == '5'):
        print(dataset)
        options()
        
    elif(op == '6'):
        print("Estatísticas idade perturbada e não perturbada:\n\nMédia: {}, {}\nMediana: {}, {}\nDesvio Padrão: {}, {}".format(np.mean(dataset['idadeCaso']), np.mean(pre_df['idadeCaso']),
                                                             np.median(dataset['idadeCaso']), np.median(pre_df['idadeCaso']),
                                                             np.std(dataset['idadeCaso']), np.std(pre_df['idadeCaso'])))
        options()
        
    else:
        return

df = pd.read_csv('covid_com_meso.csv')
print("Tamanho do dataset: ",len(df))
n = int(input('Para quantas amostras deseja executar as operações?\n'))
global dataset
global pre_df
dataset = df[:n]
dataset.drop(columns='Unnamed: 0', axis=1, inplace=True)
pre_df = dataset.copy()
options()
dataset.to_csv('covid_publico.csv')

Tamanho do dataset:  825977
Para quantas amostras deseja executar as operações?
10000


C:\Users\malu.maia\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(




############### Menu ###############

1 - Carregar dados pré-processados
2 - Supressão
3 - Generalização
4 - Perturbação
5 - Visualizar dados anonimizados
6 - Calcular estatísticas pós-perturbação
7 - Sair



Digite a operação que deseja realizar:
1
                                              localCaso   sexoCaso  \
0                CAICARA; SOBRAL; Noroeste Cearense; CE  MASCULINO   
1       BANGUE; PACAJUS; Metropolitana de Fortaleza; CE  MASCULINO   
2      ZUMBI; FORTALEZA; Metropolitana de Fortaleza; CE  MASCULINO   
3     METROPOLIS; CAUCAIA; Metropolitana de Fortalez...  MASCULINO   
4     PARANGABA; FORTALEZA; Metropolitana de Fortale...   FEMININO   
...                                                 ...        ...   
9995   DISTTRITO CARAPE; TIANGUA; Noroeste Cearense; CE   FEMININO   
9996         ZONA RURAL; TIANGUA; Noroeste Cearense; CE   FEMININO   
9997         CENTRO; IPAUMIRIM; Centro-Sul Cearense; CE  MASCULINO   
9998         ZONA RURAL; TIANGUA; Noroeste Ceare

C:\Users\malu.maia\Anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


                                              localCaso   sexoCaso  \
0                CAICARA; SOBRAL; Noroeste Cearense; CE  MASCULINO   
1       BANGUE; PACAJUS; Metropolitana de Fortaleza; CE  MASCULINO   
2      ZUMBI; FORTALEZA; Metropolitana de Fortaleza; CE  MASCULINO   
3     METROPOLIS; CAUCAIA; Metropolitana de Fortalez...  MASCULINO   
4     PARANGABA; FORTALEZA; Metropolitana de Fortale...   FEMININO   
...                                                 ...        ...   
9995   DISTTRITO CARAPE; TIANGUA; Noroeste Cearense; CE   FEMININO   
9996         ZONA RURAL; TIANGUA; Noroeste Cearense; CE   FEMININO   
9997         CENTRO; IPAUMIRIM; Centro-Sul Cearense; CE  MASCULINO   
9998         ZONA RURAL; TIANGUA; Noroeste Cearense; CE   FEMININO   
9999         ZONA RURAL; TIANGUA; Noroeste Cearense; CE   FEMININO   

     dataNascimento  idadeCaso racaCor resultadoFinalExame  
0        14/08/2003       17.0       *            Negativo  
1        07/11/1983       37.0       

C:\Users\malu.maia\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\malu.maia\Anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


                                 localCaso   sexoCaso dataNascimento  \
0              *; *; Noroeste Cearense; CE  MASCULINO     14/08/2003   
1     *; *; Metropolitana de Fortaleza; CE  MASCULINO     07/11/1983   
2     *; *; Metropolitana de Fortaleza; CE  MASCULINO     12/03/1992   
3     *; *; Metropolitana de Fortaleza; CE  MASCULINO     06/03/1970   
4     *; *; Metropolitana de Fortaleza; CE   FEMININO     10/01/1939   
...                                    ...        ...            ...   
9995           *; *; Noroeste Cearense; CE   FEMININO     15/09/1947   
9996           *; *; Noroeste Cearense; CE   FEMININO     22/08/1982   
9997         *; *; Centro-Sul Cearense; CE  MASCULINO     11/07/1987   
9998           *; *; Noroeste Cearense; CE   FEMININO     26/07/1991   
9999           *; *; Noroeste Cearense; CE   FEMININO     07/04/1991   

      idadeCaso racaCor resultadoFinalExame  
0          17.0       *            Negativo  
1          37.0       *            Negativo